$$\LARGE
\underline
{
\textbf
    {Problema 4}
}
$$

Determine los desplazamientos y rotaciones y fuerzas y torques de vínculos para el sistema de la
figura. Tome $E = 210\ GPa$ e $I = 2 x 10^{–4} m^{4}$.

<center><img src="ImgProb04G02.png"></center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import MEF as mef

In [2]:
E = 210e9
I = 2e-4  # m4
L_barra = 6  # m
k = 200e3  # N/m
P = -50e3  # N

$ \large
\underline
{\textbf{Condiciones de contorno}}
$

$\large
Nodo\ 0\ \rightarrow d_{0y}(0)=\phi_{0}(1)=0
$

$\large
Nodo\ 1\ \rightarrow d_{1y}(2)=0
$

$\large
Nodo\ 3\ (final\ del\ resorte) \rightarrow d_{3y}(6)=\phi_{3}(7)=0
$

In [3]:
MN = np.array([[0,0],[3,0],[6,0]])
MC = np.array([[0,1],[1,2]])

In [4]:
Nn, glxn = MN.shape  # cantidad de nodos, grados de lib x nodo
Ne, Nnxe = MC.shape  # cantidad de elementos, numero de nodos x elemento

In [5]:
s = np.array([0,1,2,6,7])
r = mef.vector_complemento(s, MN, glxn)

In [6]:
Us = np.array([0,0,0,0,0]).reshape([-1,1])
Fr = np.array([0,P,0]).reshape([-1,1])

In [7]:
L = L_barra/Ne  # Longitud de cada elemento

In [8]:
MR_elem = np.array([[12,6*L,-12,6*L],[6*L,4*L**2,-6*L,2*L**2],[-12,-6*L,12,-6*L],[6*L,2*L**2,-6*L,4*L**2]])
MR_elem = (E*I/L**3)*MR_elem

In [9]:
Ke = {}  # diccionario para acumular todas las K_elementales
MR = np.zeros([glxn*Nn,glxn*Nn])
archivo = 'Matrices.txt'  # Creo archivo para guardar todas las matrices elementales
with open(archivo,'w') as f:  # la f es como un alias apra el archivo que acabo de abrir
    f.write('Matrices Elementales\n ===============')
    
for e in range(Ne):
    Ke[e] = MR_elem
    fe = np.abs(Ke[e].max())
    with open('Matrices.txt','a') as f:  # 'a' de agregar
        f.write(f'\nelemento {e}, fe ={fe:4e}\n')
        f.write(f'{Ke[e]/fe}\n')

    for i in range(Nnxe):
        rangoi = np.linspace(i*glxn,(i+1)*glxn-1,glxn).astype(int)
        rangoni = np.linspace(MC[e,i]*glxn,(MC[e, i]+1)*glxn-1,glxn).astype(int)
        for j in range(Nnxe):
            rangoj = np.linspace(j*glxn,(j+1)*glxn-1,glxn).astype(int)
            rangonj = np.linspace(MC[e,j]*glxn,(MC[e, j]+1)*glxn-1,glxn).astype(int)
            MR[np.ix_(rangoni,rangonj)] += Ke[e][np.ix_(rangoi,rangoj)]

MR = np.append(MR,np.zeros([glxn*Nn,2]),1)
MR = np.append(MR,np.zeros([2,glxn*Nn+2]),0)
K_resorte = np.array([[1,-1],[-1,1]])*k
pos = np.array([4,6])
MR[np.ix_(pos,pos)] += K_resorte
fe = np.abs(MR.max())
with open('Matrices.txt','a') as f:  # 'a' de agregar
    f.write(f'\nMatriz Global, fe ={fe:4e}\n')
    f.write(f'{MR/fe}\n')

In [10]:
F, U = mef.solve1D(MR, r, s, Us, Fr)

In [12]:
np.set_printoptions(formatter={"float": lambda x: "%0.2f"% (x)})
print('\033[4m'+f'Matriz global'+'\033[0m'+'[N/m 10-6]:\n\033[93m'+ f'U =\n {MR/(10e6)} \n'+'\033[0m')
print('\033[4m'+f'\nVector fuerzas y momentos' +'\033[0m'+'[N] y [N m]:\n\033[93m'+f'F = \n{F}\n'+'\033[0m')
print('\033[4m'+f'Vector desplazamientos y angulos'+'\033[0m'+'[mm]:\n\033[93m'+ f'U =\n {U*1000} \n')

Matriz global[N/m 10-6]:
U =
 [[1.87 2.80 -1.87 2.80 0.00 0.00 0.00 0.00]
 [2.80 5.60 -2.80 2.80 0.00 0.00 0.00 0.00]
 [-1.87 -2.80 3.73 0.00 -1.87 2.80 0.00 0.00]
 [2.80 2.80 0.00 11.20 -2.80 2.80 0.00 0.00]
 [0.00 0.00 -1.87 -2.80 1.89 -2.80 -0.02 0.00]
 [0.00 0.00 2.80 2.80 -2.80 5.60 0.00 0.00]
 [0.00 0.00 0.00 0.00 -0.02 0.00 0.02 0.00]
 [0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00]] 


Vector fuerzas y momentos[N] y [N m]:
F = 
[[-69767.44]
 [-69767.44]
 [116279.07]
 [0.00]
 [-50000.00]
 [0.00]
 [3488.37]
 [0.00]]

Vector desplazamientos y angulos[mm]:
U =
 [[0.00]
 [0.00]
 [0.00]
 [-2.49]
 [-17.44]
 [-7.48]
 [0.00]
 [0.00]] 

